In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from joblib import load, dump

from src.scripts.artemis_data import get_ns_cols, get_email_cols

In [ ]:
df = pd.read_csv("../data/processed/final_whois_data.csv")

In [ ]:
label = 'community'
y = df[label]
X = df.drop(columns=label)

In [ ]:
enc_dict = load("../models/enc_dict.joblib")
country_encoder = load('../models/country_encoder.joblib')

In [ ]:
X

In [ ]:
cols_to_drop = ['domain', 'updated_date', 'expiration_date', 'creation_date', 'days_since_creation', 'malicious',
                'min_entropy', 'max_entropy', 'avg_entropy', 'domain_count', 'malicious_ratio'
                ]
cols_to_drop += get_ns_cols(7)
cols_to_drop += get_email_cols(3)
X = X.drop(columns=cols_to_drop)


In [ ]:
for col in enc_dict.keys():
    X[col] = enc_dict[col].transform(X[col])
for col in ['country', 'dns_rec_a_cc', 'dns_rec_mx_cc']:
    X[col] = country_encoder.transform(X[col])

In [ ]:
X = X.fillna(-1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
clf = KNeighborsClassifier(weights='distance').fit(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

In [ ]:
dump(clf, '../models/community_predictor.joblib')

In [ ]:
X_train